Author: Yepeng Li

# 1.Data wrangling process

In [1]:
import xml.etree.cElementTree as ET 
from collections import defaultdict 
import re 
import pprint 
import codecs 
import json 

In [2]:
OSMFILE = "Houston_texas.osm" 
CREATED = [ "version", "changeset", "timestamp", "user", "uid"] 

In [3]:
infile = OSMFILE

In [4]:
#pre-compiled regex queries 
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE) 
lower = re.compile(r'^([a-z]|_)*$') 
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$') 
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]') 

# Data overview 

In [5]:
# Identify number of tags in osm file 
def count_tags(filename):  
    #initialize defaultdict to avoid KeyError and allow new keys not found in dictionary yet 
    tags = defaultdict(int) 
    #iterate through each node element and increment the dictionary value for that node.tag key 
    for event, node in ET.iterparse(filename): 
        if event == 'end':  
            tags[node.tag]+=1 
        # discard the element is needed to clear from memory and speed up processing 
        node.clear()              
    return tags 
 
tags = count_tags('Houston_texas.osm') 
pprint.pprint(tags) 

defaultdict(<type 'int'>, {'node': 2865568, 'nd': 3410766, 'bounds': 1, 'member': 27143, 'tag': 2059263, 'relation': 2362, 'way': 344212, 'osm': 1})


In [6]:
# Identify Key Type Errors 
# Identify errors function 
# Used by count_key_kinks()
lower = re.compile(r'^([a-z]|_)*$') 
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$') 
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]') 

def key_type(element, keys): 
    if element.tag == "tag": 
        if re.search(lower, element.attrib['k']):         
            keys['lower'] += 1             
        elif re.search(lower_colon, element.attrib['k']): 
            keys['lower_colon'] += 1             
        elif re.search(problemchars, element.attrib['k']): 
            keys['problemchars'] += 1 
            #print out any values with problematic characters 
            #print element             
            print element.attrib['k']             
        else: 
            keys['other'] += 1                       
    return keys 
 
def count_key_kinks(filename): 
    #initialize dictionary 
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0} 
    for _, element in ET.iterparse(filename): 
        keys = key_type(element, keys) 
        # discard the element is needed to clear from memory and speed up processing 
        element.clear() 
    return keys  
 
keys = count_key_kinks('Houston_texas.osm') 
pprint.pprint(keys)

service area
service area
{'lower': 839457, 'lower_colon': 1164223, 'other': 55581, 'problemchars': 2}


In [7]:
# Identify unique contributors in osm file 
def unique_users(filename): 
    users = set() 
    for _, element in ET.iterparse(filename): 
        try: 
            users.add(element.attrib['uid']) 
        except KeyError: 
            pass 
        element.clear() #to clear memory 
    return users 
 
users = unique_users('Houston_texas.osm') 
print len(users) 

1293


# Data Auditting and updating

In [8]:
# List of the street types we expect to see in the US
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road",  
            "Trail", "Parkway", "Commons", "Ring Road"] 

In [9]:
# Map the 'incorrect' street abbrvs with the correct street names
# Update the mapping by audit street report

Street_mapping = {'AVE': 'Avenue',
 'AVENUE': 'Avenue',
 'Ave': 'Avenue',
 'Ave.': 'Avenue',
 'Blvd': 'Boulevard',
 'Blvd.': 'Boulevard',
 'Broadway': 'Broadway',
 'Bypass': 'Bypass',
 'Circle': 'Circle',
 'Crossing': 'Crossing',
 'Cypress': 'Cypress',
 'DRIVE': 'Drive',
 'Dr': 'Drive',
 'E': 'East',
 'East':'East',
 'Freeway':'Freeway',
 'Frwy':'Freeway',
 'Fwy': 'Freeway',
 'Highway': 'Highway',
 'Ln': 'Lane',
 'Loop': 'Loop',
 'N': 'North',
 'North':'North',
 'Pkwy': 'Parkway',
 'Rd': 'Road',
 'Rd.': 'Road',
 'Riverway': 'Riverway',
 'Road)': 'Road',
 'S': 'South',
 'ST': 'Street',
 'South': 'South',
 'Speedway': 'Speedway',
 'St': 'Street',
 'St.': 'Street',
 'Stree': 'Street',
 'Way': 'Way',
 'West': 'West',
 'blvd': 'Boulevard',
 'street': 'Street',
}

In [10]:
#Mapping postal_code according to the audit postal code report

postal_code_mapping = {'77007-2121': '77007','77005-1890': '77005', '77339-1510': '77339', 
'77027-6850': '77027', '77007-2112': '77007','77007-2113': '77007', 'TX 77009': '77009',
'77025-9998': '77025','77019-1999': '77019', '77042-9998': '77042', 'Weslayan Street': '71000',
'77054-1921': '77054', '77024-8022': '77024', '77478-': '77478', '77036-3590': '77036', '7-': '71000', 
'77077-9998': '77077','TX 77494':'77494','TX 77086':'77086','77584-':'77584','77459-':'77459',
'773867386':'77386', 
}
default_phone_number = "+" + "1 " + "8888888888"

In [11]:
# Add error street names to street_type
def audit_street_type(street_types, street_name): 
    m = street_type_re.search(street_name) 
    if m: 
        street_type = m.group() 
        if street_type not in expected:
            street_types[street_type].add(street_name) 


In [12]:
# Add error postal codes to postcodes. Form example ['adcbe', '564324', '123lf'] 
def audit_postal_code(invalid_postal_codes, postal_code): 
    try: 
        if postal_code[0]== 'u' or postal_code.isdigit() == False or len(postal_code) != 5 : 
            raise ValueError 
    except ValueError: 
        invalid_postal_codes[postal_code] += 1 

In [13]:
# Add error phone number or phone number without country code to invalid phone numbers
def audit_phone_number(invalid_phone_numbers, phone_number): 
    try: 
        if phone_number[:2] != '+1': 
            raise ValueError 
        elif len(phone_number) > 17:      # eg +1 (888)-888-888888
            raise ValueError
    except ValueError: 
        invalid_phone_numbers[phone_number] += 1 

In [14]:
def is_street_name(elem): 
    return (elem.attrib['k'] == "addr:street") 
 
def is_postal_code(elem): 
    return (elem.attrib['k'] == "addr:postcode") 

def is_phone_number(elem): 
    return (elem.attrib['k'] == "phone") 


In [15]:
def audit(osmfile): 
    osm_file = open(osmfile, "r") 
    street_types = defaultdict(set) 
    invalid_postal_codes = defaultdict(int) 
    invalid_phone_numbers = defaultdict(int) 
    for event, elem in ET.iterparse(osm_file, events=("start",)): 
        if elem.tag == "node" or elem.tag == "way": 
            for tag in elem.iter("tag"): 
                if is_street_name(tag): 
                    audit_street_type(street_types, tag.attrib['v']) 
                elif is_postal_code(tag): 
                    audit_postal_code(invalid_postal_codes, tag.attrib['v']) 
                elif is_phone_number(tag): 
                    audit_phone_number(invalid_phone_numbers, tag.attrib['v']) 
    return [invalid_postal_codes, invalid_phone_numbers, street_types] 


In [16]:
#standardizes street types with a replacement map 
def update_name(name, mapping): 
    name = name.split(' ') 
    type = name[-1] 
    if type in mapping: 
        name[-1] = mapping[type] 
        name = ' '.join(name) 
        name = name.title() 
    return name 

In [17]:
#checks if postal code within valid range, if not replaces with 11000 default    
def update_postal_code(postal_code,mapping): 
    try: 
        if len(postal_code) != 5: 
            raise ValueError 
        else: 
            return int(postal_code) 
    except ValueError: 
        type = postal_code
        if type in mapping: 
            postal_code = mapping[type] 
        return postal_code 

In [18]:
#standardizes phone number formatting 
def update_phone_number(phone_number): 
    phone_number = phone_number.translate(None, ' ()-')
    if len(phone_number) > 10:
        return default_phone_number
    else:
        phone_number = '+1 ' + phone_number
        return phone_number

In [19]:
def shape_element(e): 
    node = {} 
    node['created'] = {} 
    node['pos'] = [0,0] 
    if e.tag == "way": 
        node['node_refs'] = [] 
    if e.tag == "node" or e.tag == "way" : 
        node['type'] = e.tag 
        #attributes 
        for k, v in e.attrib.iteritems(): 
            #latitude 
            if k == 'lat': 
                try: 
                    lat = float(v) 
                    node['pos'][0] = lat 
                except ValueError: 
                    pass 
            #longitude 
            elif k == 'lon': 
                try: 
                    lon = float(v) 
                    node['pos'][1] = lon 
                except ValueError: 
                    pass 
            #creation metadata 
            elif k in CREATED: 
                node['created'][k] = v 
            else: 
                node[k] = v 
        #children 
        for tag in e.iter('tag'): 
            k = tag.attrib['k'] 
            v = tag.attrib['v'] 
            if problemchars.match(k): 
                continue 
            elif lower_colon.match(k): 
                k_split = k.split(':') 
                #address fields 
                if k_split[0] == 'addr': 
                    k_item = k_split[1] 
                    if 'address' not in node: 
                        node['address'] = {} 
                    #streets 
                    if k_item == 'street': 
                        v = update_name(v, Street_mapping)                     
                    #postal codes 
                    if k_item == 'postcode': 
                        v = update_postal_code(v,postal_code_mapping) 
                    node['address'][k_item] = v 
                    continue 
            else:                 
                #phone numbers 
                if(is_phone_number(tag)): 
                    v = update_phone_number(v) 
            node[k] = v 
        #way children 
        if e.tag == "way": 
            for n in e.iter('nd'): 
                ref = n.attrib['ref'] 
                node['node_refs'].append(ref); 
        return node 
    else: 
        return None 

In [20]:
def process_map(file_in, pretty = False): 
    file_out = "{0}.json".format(file_in) 
    data = [] 
    with codecs.open(file_out, "w") as fo: 
        for _, element in ET.iterparse(file_in): 
            el = shape_element(element) 
            if el: 
                data.append(el) 
                if pretty: 
                    fo.write(json.dumps(el, indent=2)+"\n") 
                else: 
                    fo.write(json.dumps(el) + "\n") 
    return data 

In [21]:
def audit_report(): 
    audit_data = audit(OSMFILE) 
    pprint.pprint(audit_data[0]) 
    pprint.pprint(audit_data[1]) 
    pprint.pprint(dict(audit_data[2])) 

In [22]:
audit_report() 

defaultdict(<type 'int'>, {'77007-2121': 1, '77054-1921': 1, '773345': 2, '77042-9998': 1, '77077-9998': 1, '77339-1510': 3, '77027-6850': 1, '77019-1999': 1, '77007-2112': 1, '77007-2113': 2, '7-': 1, 'TX 77009': 1, 'TX 77494': 1, '77005-1890': 1, 'TX 77086': 1, '77584-': 1, '77024-8022': 2, '773867386': 1, 'Weslayan Street': 1, '77478-': 1, '77025-9998': 1, '77036-3590': 1})
defaultdict(<type 'int'>, {'(281) 996-9967': 1, '713-472-5172': 1, '281-497-0630': 1, '(281) 809-9779': 1, '713-284-1200': 1, '713-721-0011': 1, '713-284-1204': 1, '(281) 370-2072': 1, '(281) 320-3700': 1, '(281) 717-8209': 1, '281-634-0612': 1, '281-579-3890': 1, '(281) 242-2424': 1, '(281) 693-0603': 1, '281-578-5566': 1, '(281) 351-1972': 1, '(713) 847-0071': 1, '281-367-5870': 1, '(713) 718-7800': 1, '(713) 743-4646': 1, '(281) 497-8479': 1, '(713) 278-8450': 1, '(713) 718-2000': 1, '(281) 776-0143': 1, '(713) 293-6100': 1, '713-667-1167': 1, '713 790 0099': 1, '(800) 553-2128': 1, '(713) 743-8888': 1, '(281)

In [22]:
#process data and output json file

process_map(OSMFILE, False) 

[{'created': {'changeset': '6369617',
   'timestamp': '2010-11-14T18:44:21Z',
   'uid': '9176',
   'user': 'Maarten Deen',
   'version': '2'},
  'id': '4335942',
  'pos': [29.353278, -95.487423],
  'type': 'node'},
 {'created': {'changeset': '6369617',
   'timestamp': '2010-11-14T18:44:24Z',
   'uid': '9176',
   'user': 'Maarten Deen',
   'version': '2'},
  'id': '4421932',
  'pos': [29.768133, -94.813628],
  'type': 'node'},
 {'created': {'changeset': '34502220',
   'timestamp': '2015-10-08T00:49:18Z',
   'uid': '220395',
   'user': 'LizBarry',
   'version': '15'},
  'ele': '13',
  'id': '27526178',
  'is_in:continent': 'North America',
  'is_in:country': 'United States',
  'is_in:country_code': 'US',
  'name': 'Houston',
  'name:am': u'\u1202\u12cd\u1235\u1270\u1295',
  'name:ar': u'\u0647\u064a\u0648\u0633\u062a\u0646\u060c \u062a\u0643\u0633\u0627\u0633',
  'name:az': 'Hyuston',
  'name:be': u"\u0425'\u044e\u0441\u0442\u0430\u043d",
  'name:bg': u'\u0425\u044e\u0441\u0442\u044a\u04

# Import json file into MongoDB

# 2. Data overview after cleaning

In [23]:
from pymongo import MongoClient

In [24]:
client = MongoClient()
db = client.test

In [25]:
postal_code_sort = db.Houston111.aggregate([{"$match":{"address.postcode":{"$exists":1}}},
                                          {"$group":{"_id":"$address.postcode",
                                                     "count":{"$sum":1}}}, 
                                          {"$sort":{"count":-1}}])

In [26]:
for p in postal_code_sort:
    print p

{u'count': 481, u'_id': 77096}
{u'count': 253, u'_id': 77449}
{u'count': 211, u'_id': 77401}
{u'count': 184, u'_id': 77339}
{u'count': 174, u'_id': 77494}
{u'count': 97, u'_id': 77002}
{u'count': 83, u'_id': 77079}
{u'count': 83, u'_id': 77076}
{u'count': 80, u'_id': 77396}
{u'count': 69, u'_id': 77006}
{u'count': 66, u'_id': 77338}
{u'count': 65, u'_id': 77030}
{u'count': 63, u'_id': 77090}
{u'count': 61, u'_id': 77060}
{u'count': 50, u'_id': 77032}
{u'count': 46, u'_id': 77586}
{u'count': 45, u'_id': 77070}
{u'count': 44, u'_id': 77008}
{u'count': 41, u'_id': 77084}
{u'count': 40, u'_id': 77027}
{u'count': 39, u'_id': 77024}
{u'count': 39, u'_id': 77459}
{u'count': 37, u'_id': 77479}
{u'count': 37, u'_id': 77007}
{u'count': 35, u'_id': 77204}
{u'count': 33, u'_id': 77573}
{u'count': 32, u'_id': 77489}
{u'count': 32, u'_id': 77478}
{u'count': 31, u'_id': 77345}
{u'count': 30, u'_id': 77380}
{u'count': 27, u'_id': 77004}
{u'count': 27, u'_id': 77346}
{u'count': 25, u'_id': 77022}
{u'co

In [32]:
city_sort = db.Houston111.aggregate([{"$match":{"address.city":{"$exists":1}}}, {"$group":{"_id":"$address.city", "count":{"$sum":1}}}, {"$sort":{"count":1}}])
i = 0
for c in city_sort:
    if i < 2:
        print c
        i = i +1

{u'count': 1, u'_id': u'Laks Jackson'}
{u'count': 1, u'_id': u'Plantersville'}


#Number of ddocuments,nodes and ways

In [33]:
print "The total number of documents: {0}".format(db.Houston111.find().count())
print "The total number of nodes: {0}".format(db.Houston111.find({"type" : "node"}).count())
print "The total number of ways: {0}".format(db.Houston111.find({"type" : "way"}).count())


The total number of documents: 3209780
The total number of nodes: 2865564
The total number of ways: 344195


In [34]:
number_of_unique_users = db.Houston111.aggregate([
                                          {"$group" : {
                                             "_id" : "$created.user",
                                             "count" : {"$sum" : 1}
                                                      }
                                          }])
top_5_users = db.Houston111.aggregate([
                                      {"$group" : {
                                            "_id" : "$created.user",
                                            "count" : {"$sum" : 1}
                                      }},
                                      {"$sort" : {"count" : -1}},
                                      {"$limit" : 5 }
                                     ])
five_most_popular_amenities = db.Houston111.aggregate([{"$match" : {"amenity" : {"$exists" : 1}}},
                                                      {"$group" : {"_id" : "$amenity",
                                                                  "count" : {"$sum" : 1}}
                                                      },
                                                      {"$sort" : {"count" : -1}},
                                                      {"$limit" : 5}
                                                     ])


distinct_amenities = db.Houston111.distinct("amenity")
number_of_amenities = len(distinct_amenities)
number_of_schools = int(db.Houston111.find({"amenity" : "school"}).count())
number_of_cafes = int(db.Houston111.find({"amenity" : "cafe"}).count())


#Number of Unique users in Houston, TX

In [35]:
user_count = 0
for doc in number_of_unique_users: user_count += 1
print user_count

1283


#The five most popular amenities in Houston, TX

In [36]:
for doc in five_most_popular_amenities: print doc

{u'count': 3496, u'_id': u'parking'}
{u'count': 2452, u'_id': u'place_of_worship'}
{u'count': 1649, u'_id': u'school'}
{u'count': 904, u'_id': u'fast_food'}
{u'count': 828, u'_id': u'restaurant'}


#The top 5 contributing users

In [37]:
for doc in top_5_users: print doc

{u'count': 591156, u'_id': u'woodpeck_fixbot'}
{u'count': 547958, u'_id': u'TexasNHD'}
{u'count': 425099, u'_id': u'afdreher'}
{u'count': 199984, u'_id': u'scottyc'}
{u'count': 193936, u'_id': u'cammace'}


#Numbers of schools,cafes and distinct amenities

In [38]:
print "The total number of schools: {0}".format(number_of_schools)
print "The number of cafes: {0}".format(number_of_cafes)
print "The number of distinct amenities: {0}".format(len(distinct_amenities))

The total number of schools: 1649
The number of cafes: 126
The number of distinct amenities: 98


In [44]:
# Number of users appearing only once (having 1 post)                                               
one_post_user = db.Houston111.aggregate([{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$group":{"_id":"$count", "num_users":{"$sum":1}}}, {"$sort":{"_id":1}}, {"$limit":1}])

# “_id” represents postcount
for user in one_post_user:
    print user

{u'num_users': 248, u'_id': 1}


# 3. Getting stats from dataset

# 4.Additional data exploration using MongoDB queries



In [40]:
#Biggest religion (no surprise here)
                                                
biggest_religion = db.Houston111.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"place_of_worship"}},
                                                
{"$group":{"_id":"$religion", "count":{"$sum":1}}},
                                                
{"$sort":{"count":-1}}, {"$limit":1}])

for b in biggest_religion:
    print b

{u'count': 2376, u'_id': u'christian'}


In [41]:
# Most popular cuisines
                                                
most_pop_cuisine = db.Houston111.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"restaurant"}},
                     {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},
                     {"$sort":{"count":-1}}, {"$limit":2}])
for m in most_pop_cuisine:
    print m    

{u'count': 399, u'_id': None}
{u'count': 82, u'_id': u'mexican'}


# 5.Conclusion

After this review of the data it’s obvious that the Houston metro area is incomplete, though I believe it has been well cleaned for the purposes of this exercise. It interests me to notice a fair amount of GPS data makes it into OpenStreetMap.org on account of users’ efforts, whether by scripting a map editing bot or otherwise. With a rough GPS data processor in place and working together with a more robust data processor similar to data.py I think it would be possible to input a great amount of cleaned data to OpenStreetMap.org.

Another interesting problem is that the query result about most popupar cuisine are "None"(399) and "Mexican"(82).If there is more information collected about the unknown type of cuisine it will be better for people to know more about the favarite food of Houston residents. Especially for people who want to invest a restaurant such imformtion is very important.
                                